## Project Objective


To build a model that accurately predicts the unit sales for the items sold by Corporation Favorita

## Hypothesis & Questions

### Hypotheses

### Questions

1. Is the train dataset complete (has all the required dates)?
2. Which dates have the lowest and highest sales for each year?
3. Did the earthquake impact sales?
4. Are certain groups of stores selling more products? (Cluster, city, state, type)
5. Are sales affected by promotions, oil prices and holidays?
6. What analysis can we get from the date and its extractable features?
7. What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)

# Installing Scikit-learn module

In [1]:
pip install -U scikit-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Importing Libraries

In [3]:

# importing the neccessary python libraries 
import numpy as np 
import pandas as pd 

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from  sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing

import xgboost as xgb
from xgboost import XGBRegressor

import matplotlib.pyplot as plt
%matplotlib inline 
import plotly.express as px
import seaborn as sns
#import statsmodels.api as sm 

from itertools import *

import warnings

# Hiding the warnings
warnings.filterwarnings('ignore')

print("Loading complete.", "Warnings hidden.")


Loading complete. Warnings hidden.


### Loading Train Data

In [4]:
train_data = pd.read_csv("train.csv")
train_data

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [6]:
#pd.options.display.float_format = '{:,0.2f}'.format

In [7]:
unique_days = train_data["date"].unique()
unique_days

array(['2013-01-01', '2013-01-02', '2013-01-03', ..., '2017-08-13',
       '2017-08-14', '2017-08-15'], dtype=object)

In [8]:
train_data["sales_date"] = pd.to_datetime(train_data["date"]).dt.date
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
 6   sales_date   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 160.3+ MB


In [9]:
range_0f_date = train_data.sales_date.min(),train_data.sales_date.max()
range_0f_date

(datetime.date(2013, 1, 1), datetime.date(2017, 8, 15))

In [10]:
number_of_expected_days = pd.date_range(start = train_data["sales_date"].min(),end = train_data["sales_date"].max())
number_of_expected_days

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08',
               '2013-01-09', '2013-01-10',
               ...
               '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09',
               '2017-08-10', '2017-08-11', '2017-08-12', '2017-08-13',
               '2017-08-14', '2017-08-15'],
              dtype='datetime64[ns]', length=1688, freq='D')

We note a difference of 4 days between the actual dates (1,684) and expected dates (1,688) within the range. As such we have to find the missing dates and add them to ensure completeness of the dates.

This gives the answer to question 1 (Is the train dataset complete (has all the required dates)?) as a no.

In [11]:
missing_dates = set(number_of_expected_days.date) - set()

### Hypothesis

this is the hypothesis

In [12]:
# Getting the list of unique sets 
unique_stores = train_data["store_nbr"].unique()
unique_stores

array([ 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,  2, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29,  3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,  4,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49,  5, 50, 51, 52, 53, 54,  6,
        7,  8,  9], dtype=int64)

In [13]:
# Getting unique Families 
unique_families = train_data["family"].unique()
unique_families

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

Since we're predicting the sales for each store, it means we have to fill in the missing dates for each store. We will do this with the _product_ module from _itertools_

In [14]:
missing_data = list(product(missing_dates,unique_stores,unique_families))
train_addon = pd.DataFrame(missing_data, columns = ["sales_date","store_nbr","family"])
train_addon


,sales_date,store_nbr,family
0,2015-06-25,1,AUTOMOTIVE
1,2015-06-25,1,BABY CARE
2,2015-06-25,1,BEAUTY
3,2015-06-25,1,BEVERAGES
4,2015-06-25,1,BOOKS
...,...,...,...
3008011,2014-01-18,9,POULTRY
3008012,2014-01-18,9,PREPARED FOODS
3008013,2014-01-18,9,PRODUCE
3008014,2014-01-18,9,SCHOOL AND OFFICE SUPPLIES


In [15]:
train_data = pd.concat([train_data,train_addon],ignore_index = True)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6008904 entries, 0 to 6008903
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           float64
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  float64
 6   sales_date   object 
dtypes: float64(3), int64(1), object(3)
memory usage: 320.9+ MB


In [16]:
train_data

,id,date,store_nbr,family,sales,onpromotion,sales_date
0,0.0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,2013-01-01
1,1.0,2013-01-01,1,BABY CARE,0.0,0.0,2013-01-01
2,2.0,2013-01-01,1,BEAUTY,0.0,0.0,2013-01-01
3,3.0,2013-01-01,1,BEVERAGES,0.0,0.0,2013-01-01
4,4.0,2013-01-01,1,BOOKS,0.0,0.0,2013-01-01
...,...,...,...,...,...,...,...
6008899,NaN,NaN,9,POULTRY,NaN,NaN,2014-01-18
6008900,NaN,NaN,9,PREPARED FOODS,NaN,NaN,2014-01-18
6008901,NaN,NaN,9,PRODUCE,NaN,NaN,2014-01-18
6008902,NaN,NaN,9,SCHOOL AND OFFICE SUPPLIES,NaN,NaN,2014-01-18


- With December 25 omitted from each of the years, I assume that it was deliberate - most likely because all shops are closed on December 25 each year. In effect, no items would have been on promotion and no sales would have been made; that is to say that it is safe to fill the null "sales" and "onpromotion" column data with 0.

- By this, I am also dropping the "id" column as it will not be relevant to subsequent analyses and modelling.

- I will be filling the missing dates in the original dates column with the sales data, for aesthetic purposes only.

In [17]:
# Dropping "id" and "date" columns
train_data.drop(columns = ["id", "date"], axis = 1, inplace = True)

# Filling missing rows in the sales column and casting it to numeric
train_data["sales"].fillna(0, inplace = True)
train_data["sales"] = pd.to_numeric(train_data["sales"])

# Filling missing rows in the onpromotion column
train_data["onpromotion"].fillna(0, inplace = True)

train_data

,store_nbr,family,sales,onpromotion,sales_date
0,1,AUTOMOTIVE,0.0,0.0,2013-01-01
1,1,BABY CARE,0.0,0.0,2013-01-01
2,1,BEAUTY,0.0,0.0,2013-01-01
3,1,BEVERAGES,0.0,0.0,2013-01-01
4,1,BOOKS,0.0,0.0,2013-01-01
...,...,...,...,...,...
6008899,9,POULTRY,0.0,0.0,2014-01-18
6008900,9,PREPARED FOODS,0.0,0.0,2014-01-18
6008901,9,PRODUCE,0.0,0.0,2014-01-18
6008902,9,SCHOOL AND OFFICE SUPPLIES,0.0,0.0,2014-01-18


**Transactions data**

In [18]:
transactions = pd.read_csv("transactions.csv")
transactions

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


In [19]:
# Viewing basic information about the transactions data
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


In [20]:
transactions.nunique()

date            1682
store_nbr         54
transactions    4993
dtype: int64

- Since the train data has the same number of unique stores as the transactions data, we can use the unique stores variable defined earlier to fill in the missing dates.
- Also, given that the transactions and train data cover the same period, it is concerning that the transactions data has even less unique dates than the train data has. As such, we have to find and impute the missing dates as done for the train data.

In [21]:
transactions["sales_date"] = pd.to_datetime(transactions["date"]).dt.date

In [22]:
# Getting missing dates
missing_txn_dates = set(number_of_expected_days.date) - set(transactions["sales_date"].unique())
missing_txn_dates

{datetime.date(2013, 12, 25),
 datetime.date(2014, 12, 25),
 datetime.date(2015, 12, 25),
 datetime.date(2016, 1, 1),
 datetime.date(2016, 1, 3),
 datetime.date(2016, 12, 25)}

In [23]:
missing_txn_data = list(product(missing_txn_dates, unique_stores))
txn_data_addon = pd.DataFrame(missing_txn_data, columns = ["sales_date", "store_nbr"])
txn_data_addon

,sales_date,store_nbr
0,2016-01-01,1
1,2016-01-01,10
2,2016-01-01,11
3,2016-01-01,12
4,2016-01-01,13
...,...,...
319,2016-01-03,54
320,2016-01-03,6
321,2016-01-03,7
322,2016-01-03,8


In [24]:
transactions

,date,store_nbr,transactions,sales_date
0,2013-01-01,25,770,2013-01-01
1,2013-01-02,1,2111,2013-01-02
2,2013-01-02,2,2358,2013-01-02
3,2013-01-02,3,3487,2013-01-02
4,2013-01-02,4,1922,2013-01-02
...,...,...,...,...
83483,2017-08-15,50,2804,2017-08-15
83484,2017-08-15,51,1573,2017-08-15
83485,2017-08-15,52,2255,2017-08-15
83486,2017-08-15,53,932,2017-08-15


In [25]:
# Adding the data for the missing transaction dates to the main transaction data and filling nulls with 0
transactions = pd.concat([transactions, txn_data_addon], ignore_index=True)
transactions.drop("date", axis = 1, inplace = True)
transactions["transactions"].fillna(0, inplace = True)

In [26]:
# Recasting the sales date column data type to date
transactions["sales_date"] = pd.to_datetime(transactions["sales_date"]).dt.date
transactions

,store_nbr,transactions,sales_date
0,25,770.0,2013-01-01
1,1,2111.0,2013-01-02
2,2,2358.0,2013-01-02
3,3,3487.0,2013-01-02
4,4,1922.0,2013-01-02
...,...,...,...
83807,54,0.0,2016-01-03
83808,6,0.0,2016-01-03
83809,7,0.0,2016-01-03
83810,8,0.0,2016-01-03


In [27]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83812 entries, 0 to 83811
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   store_nbr     83812 non-null  int64  
 1   transactions  83812 non-null  float64
 2   sales_date    83812 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.9+ MB


**Holidays and events data**

In [28]:
holidays_events = pd.read_csv("holidays_events.csv")
holidays_events

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [29]:
holidays_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


The holidays and events dataframe looks complete, hence there will be no need for any cleaning now.

In [30]:
holidays_events["date"] = pd.to_datetime(holidays_events["date"]).dt.date
holidays_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [31]:
holidays_events.nunique()

date           312
type             6
locale           3
locale_name     24
description    103
transferred      2
dtype: int64

In [32]:
holidays_events["type"].unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)

In [33]:
holidays_events[holidays_events["type"] == "Work Day"]

,date,type,locale,locale_name,description,transferred
42,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
43,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False
149,2014-12-20,Work Day,National,Ecuador,Recupero Puente Navidad,False
161,2015-01-10,Work Day,National,Ecuador,Recupero Puente Primer dia del ano,False
283,2016-11-12,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False


In [34]:
# Getting missing dates
missing_holiday_dates = set(number_of_expected_days.date) - set(holidays_events["date"].unique())
missing_holiday_dates

{datetime.date(2017, 7, 16),
 datetime.date(2014, 7, 28),
 datetime.date(2016, 3, 26),
 datetime.date(2015, 7, 22),
 datetime.date(2016, 2, 16),
 datetime.date(2016, 9, 25),
 datetime.date(2015, 6, 3),
 datetime.date(2013, 1, 19),
 datetime.date(2015, 10, 8),
 datetime.date(2015, 10, 21),
 datetime.date(2016, 10, 27),
 datetime.date(2017, 4, 16),
 datetime.date(2015, 6, 1),
 datetime.date(2013, 8, 27),
 datetime.date(2015, 11, 14),
 datetime.date(2013, 7, 22),
 datetime.date(2017, 6, 16),
 datetime.date(2014, 11, 30),
 datetime.date(2017, 8, 2),
 datetime.date(2017, 7, 12),
 datetime.date(2016, 1, 12),
 datetime.date(2015, 7, 31),
 datetime.date(2014, 2, 19),
 datetime.date(2013, 9, 19),
 datetime.date(2014, 11, 8),
 datetime.date(2014, 2, 14),
 datetime.date(2013, 9, 9),
 datetime.date(2016, 2, 20),
 datetime.date(2014, 1, 22),
 datetime.date(2014, 11, 21),
 datetime.date(2015, 3, 9),
 datetime.date(2014, 2, 3),
 datetime.date(2017, 5, 4),
 datetime.date(2014, 10, 11),
 datetime.date(

In [35]:
# Creating a dataframe for the missing dates in the holiday data
holidays_add = pd.DataFrame(missing_holiday_dates, columns = ["date"])
holidays_add

,date
0,2017-07-16
1,2014-07-28
2,2016-03-26
3,2015-07-22
4,2016-02-16
...,...
1427,2016-06-06
1428,2014-10-05
1429,2015-12-17
1430,2013-03-15


In [36]:
# Adding the  missing holiday dates to the main dataframe
holidays_events = pd.concat([holidays_events, holidays_add], ignore_index=True)
holidays_events["date"] = pd.to_datetime(holidays_events["date"]).dt.date
holidays_events = holidays_events.sort_values(by = ["date"], ignore_index = True)
holidays_events

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
1777,2017-12-22,Additional,National,Ecuador,Navidad-3,False
1778,2017-12-23,Additional,National,Ecuador,Navidad-2,False
1779,2017-12-24,Additional,National,Ecuador,Navidad-1,False
1780,2017-12-25,Holiday,National,Ecuador,Navidad,False


**Oil data**

In [37]:
oil_data = pd.read_csv("oil.csv")
oil_data.head(10)

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21
6,2013-01-09,93.08
7,2013-01-10,93.81
8,2013-01-11,93.60
9,2013-01-14,94.27


In [38]:
oil_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


We note about 43 missing values for oil prices in the oil data. Checks online revealed that said data were unavailable in real time, as such a forward fill method will be applied to fill the nulls and a backfill applied to fill any rows missing after that.

In [39]:
# Filling nulls with forward fill and backfill
oil_data = oil_data.ffill().bfill()
oil_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1218 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [40]:
oil_data.head(10)


,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21
6,2013-01-09,93.08
7,2013-01-10,93.81
8,2013-01-11,93.60
9,2013-01-14,94.27


In [41]:
# Converting the dates in the oil data to dates
oil_data["date"] = pd.to_datetime(oil_data["date"]).dt.date

The oil data now has no nulls, and is supposed to be complete, but we note that there are still some missing dates. e.g. it moves from January 4, 2013 to January 7, 2013. A quick check reveals that those dates are weekends, implying that the data is for business days and does not include weekends. With this in mind, I assume that oil prices, for the period, are frozen at close of business days of Friday and so remain constant over the weekends. As such, the "missing dates" (weekends) can be brought in another forward fills applied to them.

In [42]:
# Getting missing dates
missing_oil_dates = set(number_of_expected_days.date) - set(oil_data["date"].unique())
missing_oil_dates

{datetime.date(2013, 1, 5),
 datetime.date(2013, 1, 6),
 datetime.date(2013, 1, 12),
 datetime.date(2013, 1, 13),
 datetime.date(2013, 1, 19),
 datetime.date(2013, 1, 20),
 datetime.date(2013, 1, 26),
 datetime.date(2013, 1, 27),
 datetime.date(2013, 2, 2),
 datetime.date(2013, 2, 3),
 datetime.date(2013, 2, 9),
 datetime.date(2013, 2, 10),
 datetime.date(2013, 2, 16),
 datetime.date(2013, 2, 17),
 datetime.date(2013, 2, 23),
 datetime.date(2013, 2, 24),
 datetime.date(2013, 3, 2),
 datetime.date(2013, 3, 3),
 datetime.date(2013, 3, 9),
 datetime.date(2013, 3, 10),
 datetime.date(2013, 3, 16),
 datetime.date(2013, 3, 17),
 datetime.date(2013, 3, 23),
 datetime.date(2013, 3, 24),
 datetime.date(2013, 3, 30),
 datetime.date(2013, 3, 31),
 datetime.date(2013, 4, 6),
 datetime.date(2013, 4, 7),
 datetime.date(2013, 4, 13),
 datetime.date(2013, 4, 14),
 datetime.date(2013, 4, 20),
 datetime.date(2013, 4, 21),
 datetime.date(2013, 4, 27),
 datetime.date(2013, 4, 28),
 datetime.date(2013, 5, 

In [43]:
oil_dates_add = pd.DataFrame(missing_oil_dates, columns = ["date"])
oil_dates_add

,date
0,2017-07-16
1,2016-03-26
2,2014-07-13
3,2016-09-25
4,2013-01-19
...,...
477,2014-10-05
478,2014-09-14
479,2017-04-01
480,2013-02-16


In [44]:
# Adding the  missing oil dates to the main dataframe
oil_data = pd.concat([oil_data, oil_dates_add], ignore_index=True)
oil_data["date"] = pd.to_datetime(oil_data["date"])
oil_data = oil_data.sort_values(by = ["date"], ignore_index = True)
oil_data.head(10)

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-05,NaN
5,2013-01-06,NaN
6,2013-01-07,93.20
7,2013-01-08,93.21
8,2013-01-09,93.08
9,2013-01-10,93.81


In [45]:
# Filling nulls with forward fill and backfill
oil_data = oil_data.ffill().bfill()
oil_data.head(10)

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-05,93.12
5,2013-01-06,93.12
6,2013-01-07,93.20
7,2013-01-08,93.21
8,2013-01-09,93.08
9,2013-01-10,93.81


In [46]:
# Recasting the oil data dates to datetime dates
oil_data["date"] = pd.to_datetime(oil_data["date"]).dt.date
oil_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1700 non-null   object 
 1   dcoilwtico  1700 non-null   float64
dtypes: float64(1), object(1)
memory usage: 26.7+ KB


**Stores data**

In [47]:
stores_data = pd.read_csv("stores.csv")
stores_data.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [48]:
stores_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


**Test Data**

In [49]:
test_data = pd.read_csv("test.csv")
test_data

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [50]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


The test data looks complete, with no nulls. Casting the date column to date will be the only cleaning activity here.

In [51]:
# Casting the date column to date data type
test_data["date"] = pd.to_datetime(test_data["date"]).dt.date

**Sample Submission**

In [52]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,0.0
28510,3029398,0.0


No changes will be made to the sample submission as it is only a guide.

## Answering the other questions

**Which dates have the lowest and highest sales for each year?**

The imputation of the originally missing dates means that automatically, minimum sales for each of the four years will be on those dates (December 25 each year), but that is not what we want. What we want to know is which days had the least sales when stores were opened, as such I will only include sales values greater than 0.

In [53]:
# Aggregating sales by dates
train_by_date = train_data[train_data["sales"] != 0.00
                          ].groupby(by = "sales_date").sales.agg(["sum"]
                                                                ).sort_values(by = "sales_date")
train_by_date

,sum
sales_date,
2013-01-01,2511.618999
2013-01-02,496092.417944
2013-01-03,361461.231124
2013-01-04,354459.677093
2013-01-05,477350.121229
...,...
2017-08-11,826373.722022
2017-08-12,792630.535079
2017-08-13,865639.677471


In [54]:
# Creating a column for the years for grouping
train_by_date["year"] = pd.to_datetime(train_by_date.index).year
train_by_date.rename(columns = {"sum":"total_sales"}, inplace = True)
train_by_date = train_by_date.reset_index()
train_by_date

,sales_date,total_sales,year
0,2013-01-01,2511.618999,2013
1,2013-01-02,496092.417944,2013
2,2013-01-03,361461.231124,2013
3,2013-01-04,354459.677093,2013
4,2013-01-05,477350.121229,2013
...,...,...,...
1679,2017-08-11,826373.722022,2017
1680,2017-08-12,792630.535079,2017
1681,2017-08-13,865639.677471,2017
1682,2017-08-14,760922.406081,2017


In [55]:
fig = px.line(train_by_date, x = "sales_date", y = "total_sales", 
              title= "Sales trend for Corporation Favorita from 2013 - 2017", 
             labels = {"sales_date":"Sales Date", "total_sales":"Total Sales"})
fig.show()

In [56]:
data_2013 = train_by_date[train_by_date["year"] == 2013]
data_2013 = data_2013.reset_index()
data_2013

,index,sales_date,total_sales,year
0,0,2013-01-01,2511.618999,2013
1,1,2013-01-02,496092.417944,2013
2,2,2013-01-03,361461.231124,2013
3,3,2013-01-04,354459.677093,2013
4,4,2013-01-05,477350.121229,2013
...,...,...,...,...
359,359,2013-12-27,479314.968043,2013
360,360,2013-12-28,556952.305979,2013
361,361,2013-12-29,499719.504924,2013
362,362,2013-12-30,635134.735851,2013


In [57]:
min_sales_13 = data_2013["total_sales"].min()
max_sales_13 = data_2013["total_sales"].max()
low_hi_sales_13 = data_2013[(data_2013["total_sales"] == min_sales_13) | (data_2013["total_sales"] == max_sales_13)]
low_hi_sales_13

,index,sales_date,total_sales,year
0,0,2013-01-01,2511.618999,2013
356,356,2013-12-23,792865.284427,2013


In [58]:
data_2014 = train_by_date[train_by_date["year"] == 2014]
data_2014 = data_2014.reset_index()
data_2014

,index,sales_date,total_sales,year
0,364,2014-01-01,8602.065404,2014
1,365,2014-01-02,801011.226041,2014
2,366,2014-01-03,680672.845603,2014
3,367,2014-01-04,936628.886604,2014
4,368,2014-01-05,949618.788940,2014
...,...,...,...,...
359,723,2014-12-27,740596.158932,2014
360,724,2014-12-28,716329.635071,2014
361,725,2014-12-29,773998.401175,2014
362,726,2014-12-30,912970.533204,2014


In [59]:
fig = px.line(data_2014, x = "sales_date", y = "total_sales", title="Sales trend for Corporation Favorita in 2014", 
             labels = {"sales_date":"Sales Date", "total_sales":"Total Sales"})
fig.show()

In [60]:
min_sales_14 = data_2014["total_sales"].min()
max_sales_14 = data_2014["total_sales"].max()
low_hi_sales_14 = data_2014[(data_2014["total_sales"] == min_sales_14) | (data_2014["total_sales"] == max_sales_14)]
low_hi_sales_14

,index,sales_date,total_sales,year
0,364,2014-01-01,8.602065e+03,2014
356,720,2014-12-23,1.064978e+06,2014


In [61]:
data_2015 = train_by_date[train_by_date["year"] == 2015]
data_2015 = data_2015.reset_index()
data_2015

,index,sales_date,total_sales,year
0,728,2015-01-01,1.277362e+04,2015
1,729,2015-01-02,6.577634e+05,2015
2,730,2015-01-03,6.488807e+05,2015
3,731,2015-01-04,7.309238e+05,2015
4,732,2015-01-05,5.692673e+05,2015
...,...,...,...,...
359,1087,2015-12-27,8.377141e+05,2015
360,1088,2015-12-28,7.896849e+05,2015
361,1089,2015-12-29,8.707620e+05,2015
362,1090,2015-12-30,1.030044e+06,2015


In [62]:
fig = px.line(data_2015, x = "sales_date", y = "total_sales", title="Sales trend for Corporation Favorita in 2015", 
             labels = {"sales_date":"Sales Date", "total_sales":"Total Sales"})
fig.show()

In [63]:
min_sales_15 = data_2015["total_sales"].min()
max_sales_15 = data_2015["total_sales"].max()
low_hi_sales_15 = data_2015[(data_2015["total_sales"] == min_sales_15) | (data_2015["total_sales"] == max_sales_15)]
low_hi_sales_15

,index,sales_date,total_sales,year
0,728,2015-01-01,1.277362e+04,2015
276,1004,2015-10-04,1.234131e+06,2015


In [64]:
data_2016 = train_by_date[train_by_date["year"] == 2016]
data_2016 = data_2016.reset_index()
data_2016

,index,sales_date,total_sales,year
0,1092,2016-01-01,1.643339e+04,2016
1,1093,2016-01-02,1.066677e+06,2016
2,1094,2016-01-03,1.226736e+06,2016
3,1095,2016-01-04,9.559569e+05,2016
4,1096,2016-01-05,8.353204e+05,2016
...,...,...,...,...
360,1452,2016-12-27,8.424755e+05,2016
361,1453,2016-12-28,9.515337e+05,2016
362,1454,2016-12-29,8.941082e+05,2016
363,1455,2016-12-30,1.163643e+06,2016


In [65]:
fig = px.line(data_2016, x = "sales_date", y = "total_sales", title="Sales trend for Corporation Favorita in 2016", 
             labels = {"sales_date":"Sales Date", "total_sales":"Total Sales"})
fig.show()

In [66]:
min_sales_16 = data_2016["total_sales"].min()
max_sales_16 = data_2016["total_sales"].max()
low_hi_sales_16 = data_2016[(data_2016["total_sales"] == min_sales_16) | (data_2016["total_sales"] == max_sales_16)]
low_hi_sales_16

,index,sales_date,total_sales,year
0,1092,2016-01-01,1.643339e+04,2016
108,1200,2016-04-18,1.345921e+06,2016


In [67]:
data_2017 = train_by_date[train_by_date["year"] == 2017]
data_2017 = data_2017.reset_index()
data_2017

,index,sales_date,total_sales,year
0,1457,2017-01-01,1.208250e+04,2017
1,1458,2017-01-02,1.402306e+06,2017
2,1459,2017-01-03,1.104377e+06,2017
3,1460,2017-01-04,9.900935e+05,2017
4,1461,2017-01-05,7.776210e+05,2017
...,...,...,...,...
222,1679,2017-08-11,8.263737e+05,2017
223,1680,2017-08-12,7.926305e+05,2017
224,1681,2017-08-13,8.656397e+05,2017
225,1682,2017-08-14,7.609224e+05,2017


In [68]:
min_sales_17 = data_2017["total_sales"].min()
max_sales_17 = data_2017["total_sales"].max()
low_hi_sales_17 = data_2017[(data_2017["total_sales"] == min_sales_17) | (data_2017["total_sales"] == max_sales_17)]
low_hi_sales_17

,index,sales_date,total_sales,year
0,1457,2017-01-01,1.208250e+04,2017
90,1547,2017-04-01,1.463084e+06,2017


In [69]:
# Combining the highest and lowest sales dates 
low_hi_sales_df = pd.concat([low_hi_sales_13, low_hi_sales_14, low_hi_sales_15, 
                             low_hi_sales_16, low_hi_sales_17], ignore_index = True)
low_hi_sales_df

,index,sales_date,total_sales,year
0,0,2013-01-01,2.511619e+03,2013
1,356,2013-12-23,7.928653e+05,2013
2,364,2014-01-01,8.602065e+03,2014
3,720,2014-12-23,1.064978e+06,2014
4,728,2015-01-01,1.277362e+04,2015
5,1004,2015-10-04,1.234131e+06,2015
6,1092,2016-01-01,1.643339e+04,2016
7,1200,2016-04-18,1.345921e+06,2016
8,1457,2017-01-01,1.208250e+04,2017
9,1547,2017-04-01,1.463084e+06,2017


The table above summarizes the dates which had least and most sales for each year. We note that Corporation Favorita made least sales on January 1 each year. For 2013 and 2014, they made most sales in December, while they made most sales in April 2016 and 2017. The outsider is 2015, when they made most sales in October.

**Did the earthquake impact sales?**

Given the foregoing - that 2016 sales peaked in April (18th April) - it is safe to assume that the magnitude 7.8 earthquake that struck Ecuador on April 16, 2016 affected sales positively.

The sales for 2016 will be visualized to give a closer look at the effects of the earthquake on the sales of Corporation Favorita.

In [70]:
# Visualizing the sales trend in 2016
fig = px.line(data_2016, x = "sales_date", y = "total_sales", title="Sales trend for Corporation Favorita in 2016", 
             labels = {"sales_date":"Sales Date", "total_sales":"Total Sales"})
fig.show()

From the graph above, we note that but for the earthquake sales would have peaked in December as it had in 2014 and 2013. But this time, it peaked in April. Sales hit USD 1.2m on April 17, the day after the earthquake and rose further to it's peak - USD 1.35m before normalizing.

For context, it is worthy of note that in all the previous years, sales would usually dip in April and May before picking up in June and the second half of the year.

Therefore, we can conclude that the earthquake impacted sales in 2016.

**Are certain groups of stores selling more products? (Cluster, city, state, type)**

To perform these analyses, the respective dataframes containing the data will have to be joined to the main train dataframe. This will be done in parts to prevent any memory errors.

***Analysis of sales by store cluster***

In [71]:
train_data.head(10)

,store_nbr,family,sales,onpromotion,sales_date
0,1,AUTOMOTIVE,0.0,0.0,2013-01-01
1,1,BABY CARE,0.0,0.0,2013-01-01
2,1,BEAUTY,0.0,0.0,2013-01-01
3,1,BEVERAGES,0.0,0.0,2013-01-01
4,1,BOOKS,0.0,0.0,2013-01-01
5,1,BREAD/BAKERY,0.0,0.0,2013-01-01
6,1,CELEBRATION,0.0,0.0,2013-01-01
7,1,CLEANING,0.0,0.0,2013-01-01
8,1,DAIRY,0.0,0.0,2013-01-01
9,1,DELI,0.0,0.0,2013-01-01


In [72]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6008904 entries, 0 to 6008903
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   store_nbr    int64  
 1   family       object 
 2   sales        float64
 3   onpromotion  float64
 4   sales_date   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 229.2+ MB


In [73]:
stores_data.head(10)

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


In [74]:
# How many unique values are there in the stores data
stores_data.nunique()

store_nbr    54
city         22
state        16
type          5
cluster      17
dtype: int64

In [75]:
# How many stores are there per cluster
cluster_check = stores_data.groupby(by = "cluster").store_nbr.agg(["count"])
cluster_check = cluster_check.reset_index()
cluster_check["cluster"] = (cluster_check["cluster"]).apply(str)
cluster_check

,cluster,count
0,1,3
1,2,2
2,3,7
3,4,3
4,5,1
5,6,6
6,7,2
7,8,3
8,9,2
9,10,6


Key notes:
- Corporation Favorita has stores in 22 different cities across 16 states.
- The stores are grouped into 5 types and 17 clusters.
- Cluster 3 has the most stores (7), followed by clusters 6 and 10 with 6 stores each. The top 5 is completed by Cluster 15 with 5 stores, and clusters 13 and 14 with 4 stores each.

Does the number of stores in a cluster affect sales? Let's find out!

*Since the shared column in both dataframes is the store_nbr, we join on that column and aggregate by cluster.*

In [76]:
# Joining the train data with the stores data
train_by_cluster = pd.merge(train_data, stores_data, on = "store_nbr")
train_by_cluster["year"] = pd.to_datetime(train_by_cluster["sales_date"]).dt.year
train_by_cluster["year"] = (train_by_cluster["year"]).apply(str)
train_by_cluster

,store_nbr,family,sales,onpromotion,sales_date,city,state,type,cluster,year
0,1,AUTOMOTIVE,0.0,0.0,2013-01-01,Quito,Pichincha,D,13,2013
1,1,BABY CARE,0.0,0.0,2013-01-01,Quito,Pichincha,D,13,2013
2,1,BEAUTY,0.0,0.0,2013-01-01,Quito,Pichincha,D,13,2013
3,1,BEVERAGES,0.0,0.0,2013-01-01,Quito,Pichincha,D,13,2013
4,1,BOOKS,0.0,0.0,2013-01-01,Quito,Pichincha,D,13,2013
...,...,...,...,...,...,...,...,...,...,...
6008899,9,POULTRY,0.0,0.0,2014-01-18,Quito,Pichincha,B,6,2014
6008900,9,PREPARED FOODS,0.0,0.0,2014-01-18,Quito,Pichincha,B,6,2014
6008901,9,PRODUCE,0.0,0.0,2014-01-18,Quito,Pichincha,B,6,2014
6008902,9,SCHOOL AND OFFICE SUPPLIES,0.0,0.0,2014-01-18,Quito,Pichincha,B,6,2014


In [77]:
# Aggregating the combined stores and training data by cluster
sales_by_cluster = train_by_cluster.groupby(by = ["cluster"]).sales.agg(["sum"])
sales_by_cluster = sales_by_cluster.reset_index().rename(columns = {"sum":"total_sales"})
sales_by_cluster["cluster"] = (sales_by_cluster["cluster"]).apply(str)

# Joining with the count of stores per cluster
sales_by_cluster = sales_by_cluster.merge(cluster_check).rename(columns = {"count":"store_count"})
sales_by_cluster

,cluster,total_sales,store_count
0,1,5.437675e+07,3
1,2,2.891640e+07,2
2,3,7.562870e+07,7
3,4,4.944344e+07,3
4,5,6.208755e+07,1
5,6,1.142544e+08,6
6,7,1.547812e+07,2
7,8,1.079282e+08,3
8,9,3.056108e+07,2
9,10,8.532443e+07,6


In [78]:
# Visualizing the total sales performance over the period by cluster
fig = px.bar(sales_by_cluster, x = "cluster", y = "total_sales",
              title="Sales Performance of Corporation Favorita's Store Clusters",
             labels = {"cluster":"Cluster", "total_sales":"Total Sales"})
fig.show()

For total sales performance over the period, it is evident from the graph above that, cluster 14 was the best cluster with USD 157.43m. Cluster 6 came second with USD 114.25m, followed by Cluster 8 (USD 107.93m) and Cluster 11 (USD 100.61m) to complete the top 4. These stores crossed the USD 100m sales mark. Cluster 10 (USD 85.32) followed in the distance.

***Was total sales per cluster influenced by the number of stores in the cluster?***

In [79]:
# Visualizing the total sales performance over the period by cluster, coloured by number of stores per cluster
sales_by_cluster["store_count"] = sales_by_cluster["store_count"].apply(str)

fig = px.bar(sales_by_cluster, x = "cluster", y = "total_sales", color = "store_count",
             category_orders={"cluster": ["1", "2", "3", "4", "5", "6", 
                                          "7", "8", "9", "10", "11", "12", 
                                          "13", "14", "15", "16", "17"],
                              "store_count": ["1", "2", "3", "4", "5", "6", "7"]},
              title="Sales Performance of Corporation Favorita's Store Clusters",
             labels = {"cluster":"Cluster", "total_sales":"Total Sales", "store_count":"Stores in Cluster"})
fig.show()

From the graph above, we note that despite having the most number of stores, Cluster 3 was not in the top 5 clusters with most total sales. This is supported by Cluster 15's (5 stores) poor ranking in terms of total sales performance. 

Given that clusters 6 and 10, each with 6 stores, were both part of the top 5 clusters with most total sales, it becomes unclear if total sales per cluster was influenced by the numberof stores in the cluster.

In [80]:
#isualizing the relationship between total sales performance and the number of stores per cluster
fig = px.scatter(sales_by_cluster, x = "store_count", y = "total_sales", trendline = "ols",
             category_orders={"cluster": ["1", "2", "3", "4", "5", "6", "7", "8", "9", 
                                          "10", "11", "12", "13", "14", "15", "16", "17"],
                              "store_count": ["1", "2", "3", "4", "5", "6", "7"]},
              title="Sales Performance of Corporation Favorita's Store Clusters",
             labels = {"cluster":"Cluster", "total_sales":"Total Sales", "store_count":"Stores in Cluster"})
fig.show()

***Analysis of sales by city***

*How does the sales performance by city look like?*

In [81]:
# Joining the train data with the stores data
train_x_stores = pd.merge(train_data, stores_data, on = "store_nbr")
train_x_stores

,store_nbr,family,sales,onpromotion,sales_date,city,state,type,cluster
0,1,AUTOMOTIVE,0.0,0.0,2013-01-01,Quito,Pichincha,D,13
1,1,BABY CARE,0.0,0.0,2013-01-01,Quito,Pichincha,D,13
2,1,BEAUTY,0.0,0.0,2013-01-01,Quito,Pichincha,D,13
3,1,BEVERAGES,0.0,0.0,2013-01-01,Quito,Pichincha,D,13
4,1,BOOKS,0.0,0.0,2013-01-01,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...
6008899,9,POULTRY,0.0,0.0,2014-01-18,Quito,Pichincha,B,6
6008900,9,PREPARED FOODS,0.0,0.0,2014-01-18,Quito,Pichincha,B,6
6008901,9,PRODUCE,0.0,0.0,2014-01-18,Quito,Pichincha,B,6
6008902,9,SCHOOL AND OFFICE SUPPLIES,0.0,0.0,2014-01-18,Quito,Pichincha,B,6


In [82]:
# Grouping the combined dataframe by city
train_by_city = train_x_stores.groupby(by = "city").sales.agg(["sum"]).reset_index().sort_values(by = "sum", ascending = False)
train_by_city

,city,sum
18,Quito,5.567418e+08
8,Guayaquil,1.229673e+08
3,Cuenca,4.916860e+07
0,Ambato,4.030440e+07
21,Santo Domingo,3.583432e+07
13,Machala,3.338680e+07
2,Cayambe,2.832556e+07
10,Latacunga,2.118226e+07
4,Daule,1.918802e+07
12,Loja,1.886000e+07


In [83]:
# Grouping the combined dataframe by city
train_by_city = train_x_stores.groupby(by = "city").sales.agg(["sum"]).reset_index().sort_values(by = "sum", ascending = False)
train_by_city

,city,sum
18,Quito,5.567418e+08
8,Guayaquil,1.229673e+08
3,Cuenca,4.916860e+07
0,Ambato,4.030440e+07
21,Santo Domingo,3.583432e+07
13,Machala,3.338680e+07
2,Cayambe,2.832556e+07
10,Latacunga,2.118226e+07
4,Daule,1.918802e+07
12,Loja,1.886000e+07


In [84]:
# Finding the number of stores per city
stores_per_city = stores_data.groupby(by = "city").store_nbr.agg(["count"]).reset_index()
stores_per_city

,city,count
0,Ambato,2
1,Babahoyo,1
2,Cayambe,1
3,Cuenca,3
4,Daule,1
5,El Carmen,1
6,Esmeraldas,1
7,Guaranda,1
8,Guayaquil,8
9,Ibarra,1


In [85]:
# Joining the dataframe on sales per store with the dataframe with number of stores per city
train_by_city_comp = pd.merge(train_by_city, stores_per_city)
train_by_city_comp.head(10)

,city,sum,count
0,Quito,5.567418e+08,18
1,Guayaquil,1.229673e+08,8
2,Cuenca,4.916860e+07,3
3,Ambato,4.030440e+07,2
4,Santo Domingo,3.583432e+07,3
5,Machala,3.338680e+07,2
6,Cayambe,2.832556e+07,1
7,Latacunga,2.118226e+07,2
8,Daule,1.918802e+07,1
9,Loja,1.886000e+07,1


In [86]:
# Calculating an average total sales per store per city
train_by_city_comp["mean_sales"] = train_by_city_comp["sum"]/train_by_city_comp["count"]
train_by_city_comp.head(10)

,city,sum,count,mean_sales
0,Quito,5.567418e+08,18,3.093010e+07
1,Guayaquil,1.229673e+08,8,1.537091e+07
2,Cuenca,4.916860e+07,3,1.638953e+07
3,Ambato,4.030440e+07,2,2.015220e+07
4,Santo Domingo,3.583432e+07,3,1.194477e+07
5,Machala,3.338680e+07,2,1.669340e+07
6,Cayambe,2.832556e+07,1,2.832556e+07
7,Latacunga,2.118226e+07,2,1.059113e+07
8,Daule,1.918802e+07,1,1.918802e+07
9,Loja,1.886000e+07,1,1.886000e+07


In [87]:
# Calculating the average sales per city
train_by_city_avg = train_by_city_comp.sort_values(by = "mean_sales", ascending = False)
train_by_city_avg.head(10)

,city,sum,count,mean_sales
0,Quito,5.567418e+08,18,3.093010e+07
6,Cayambe,2.832556e+07,1,2.832556e+07
3,Ambato,4.030440e+07,2,2.015220e+07
8,Daule,1.918802e+07,1,1.918802e+07
9,Loja,1.886000e+07,1,1.886000e+07
10,Babahoyo,1.774687e+07,1,1.774687e+07
5,Machala,3.338680e+07,2,1.669340e+07
11,Esmeraldas,1.639169e+07,1,1.639169e+07
2,Cuenca,4.916860e+07,3,1.638953e+07
1,Guayaquil,1.229673e+08,8,1.537091e+07


In [88]:
# Visualizing the total sales performance over the period by city
train_by_city_comp["count"] = train_by_city_comp["count"].apply(str)

fig = px.bar(train_by_city_comp, x = "city", y = "sum", title="Total Sales Performance by City",
             labels = {"city":"City", "sum":"Total Sales", "count":"Stores in City"})
fig.show()

In [89]:
# Visualizing the average sales performance over the period by city
fig = px.bar(train_by_city_avg, x = "city", y = "mean_sales",
              title="Performance by City: Average Sales per Store",
             labels = {"city":"City", "mean_sales":"Average Sales", "count":"Stores in City"})
fig.show()

Looking at the total sales per city, Quito (USD 556.74m), Guayaquil (USD 122.97m), Cuenca (USD 49.17m), Ambato (USD 40.30m), and Santo Domingo (USD 35.83m) are the best performing cities. This, however, may be misleading when you look at the average sales per city.

Using the average sales per city, Quito (USD 30.93m) still leads but this time it is followed by Cayambe (USD 28.33m), Ambato (USD 20.15m), Daule (19.19m) and Loja (USD 18.86m) to make the top 5. 

Quito's superb performance both overall and on average may be largely due to it's numerical advantage, but why can same not be said for Guayaquil which has 8 stores with an average of USD 15.37m per store? The same question can be asked of Cuenca, which placed 9th by average sales with USD 16.39m, Santa Domingo (14th), and Machala (7th).

With these in mind, it is recommended that - subject to further research and considerations - Corporation Favorita should consider opening more branches in larger cities which have higher average sales per city so as to maximize the numerical advantage as is done by Quito.

*What is the relationship between the number of stores and the total sales per city?*

In [90]:
# Visualizing the relationship between the number of stores per city and the total sales per city
train_by_city_comp["count"] = train_by_city_comp["count"].apply(int)

fig = px.scatter(train_by_city_comp, x = "count", y = "sum", trendline = "ols", 
                 title = "Relationship between number of stores and total sales per city",
             labels = {"city":"City", "sum":"Total Sales", "count":"Stores in City"})
fig.show()

With an R2 value of 0.9519, the number of stores can be said to have a strong positive relationship with the total sales per city. But looking at the plot above, we note that there is an obvious outlier, which is likely to have had an impact on the trendline.

To get a better view of the relationship, we will exclude the city with 18 stores from the dataframe and then re-plot to see a "truer relationship" 

In [91]:
# Visualizing the relationship between the number of stores per city and the total sales per city without the "outlier"
fig = px.scatter(train_by_city_comp[1:], x = "count", y = "sum", trendline = "ols", 
                 title = "Relationship between number of stores and total sales per city",
             labels = {"city":"City", "sum":"Total Sales", "count":"Stores in City"})
fig.show()

Here also we see that the city with 8 stores is an outlier on the plot, so despite having an R2 of 0.9230 (strong positive relationship between the variables) we will also exclude the city with 8 stores from the plot to assess the relationship.

In [92]:
# Visualizing the relationship between the number of stores per city and the total sales per city without the two "outliers"
fig = px.scatter(train_by_city_comp[2:], x = "count", y = "sum", trendline = "ols", 
                 title = "Relationship between number of stores and total sales per city",
             labels = {"city":"City", "sum":"Total Sales", "count":"Stores in City"})
fig.show()

With the exclusion of the two outliers, the "final" plot still shows a positive correlation between the number of stores and the  total sales per city. It has an R-squared value of 0.6352, indicating a fair positive relationship between the variables.

***Analysis of sales by state***

*How does the sales performance by state look like?*

In [93]:
# Grouping the combined dataframe by state
train_by_state = train_x_stores.groupby(by = "state").sales.agg(["sum"]
                                                               ).sort_values(by = "sum", ascending = False).reset_index()
train_by_state

,state,sum
0,Pichincha,5.850674e+08
1,Guayas,1.651544e+08
2,Azuay,4.916860e+07
3,Tungurahua,4.030440e+07
4,Santo Domingo de los Tsachilas,3.583432e+07
5,El Oro,3.338680e+07
6,Los Rios,3.192537e+07
7,Manabi,2.496982e+07
8,Cotopaxi,2.118226e+07
9,Loja,1.886000e+07


In [94]:
# Finding the number of stores per sta
stores_per_state = stores_data.groupby(by = "state").store_nbr.agg(["count"]).reset_index()
stores_per_state

,state,count
0,Azuay,3
1,Bolivar,1
2,Chimborazo,1
3,Cotopaxi,2
4,El Oro,2
5,Esmeraldas,1
6,Guayas,11
7,Imbabura,1
8,Loja,1
9,Los Rios,2


In [95]:
# Joining the dataframe on sales per store with the dataframe with number of stores per state
train_by_state_comp = pd.merge(train_by_state, stores_per_state)
train_by_state_comp

,state,sum,count
0,Pichincha,5.850674e+08,19
1,Guayas,1.651544e+08,11
2,Azuay,4.916860e+07,3
3,Tungurahua,4.030440e+07,2
4,Santo Domingo de los Tsachilas,3.583432e+07,3
5,El Oro,3.338680e+07,2
6,Los Rios,3.192537e+07,2
7,Manabi,2.496982e+07,3
8,Cotopaxi,2.118226e+07,2
9,Loja,1.886000e+07,1


In [96]:
# Calculating an average total sales per store per city
train_by_state_comp["avg_per_store"] = train_by_state_comp["sum"]/train_by_state_comp["count"]
train_by_state_comp

,state,sum,count,avg_per_store
0,Pichincha,5.850674e+08,19,3.079302e+07
1,Guayas,1.651544e+08,11,1.501404e+07
2,Azuay,4.916860e+07,3,1.638953e+07
3,Tungurahua,4.030440e+07,2,2.015220e+07
4,Santo Domingo de los Tsachilas,3.583432e+07,3,1.194477e+07
5,El Oro,3.338680e+07,2,1.669340e+07
6,Los Rios,3.192537e+07,2,1.596269e+07
7,Manabi,2.496982e+07,3,8.323273e+06
8,Cotopaxi,2.118226e+07,2,1.059113e+07
9,Loja,1.886000e+07,1,1.886000e+07


In [97]:
# Calculating the average sales per state
train_by_state_avg = train_by_state_comp.sort_values(by = "avg_per_store", ascending = False)
train_by_state_avg

,state,sum,count,avg_per_store
0,Pichincha,5.850674e+08,19,3.079302e+07
3,Tungurahua,4.030440e+07,2,2.015220e+07
9,Loja,1.886000e+07,1,1.886000e+07
5,El Oro,3.338680e+07,2,1.669340e+07
10,Esmeraldas,1.639169e+07,1,1.639169e+07
2,Azuay,4.916860e+07,3,1.638953e+07
6,Los Rios,3.192537e+07,2,1.596269e+07
1,Guayas,1.651544e+08,11,1.501404e+07
11,Bolivar,1.304951e+07,1,1.304951e+07
4,Santo Domingo de los Tsachilas,3.583432e+07,3,1.194477e+07


In [98]:
# Visualizing the total sales performance over the period by state
train_by_state_comp["count"] = train_by_state_comp["count"].apply(str)

fig = px.bar(train_by_state_comp, x = "state", y = "sum", title="Total Sales Performance by State",
             labels = {"state":"State", "sum":"Total Sales"})
fig.show()

In [99]:
# Visualizing the average sales performance over the period by state
fig = px.bar(train_by_state_avg, x = "state", y = "avg_per_store",
              title="Performance by State: Average Sales per Store",
             labels = {"state":"State", "avg_per_store":"Average Sales"})
fig.show()

*What is the relationship between the number of stores and the total sales per state?*

In [100]:
# Visualizing the relationship between the number of stores and the total sales per state
train_by_state_comp["count"] = train_by_state_comp["count"].apply(int)

fig = px.scatter(train_by_state_comp, x = "count", y = "sum", trendline = "ols", 
                 title = "Relationship between number of stores and total sales per state",
             labels = {"state":"State", "sum":"Total Sales", "count":"Stores in State"})
fig.show()

With an R2 value of 0.9267, the number of stores can be said to have a strong positive relationship with the total sales per state. But the plot above, shows an outlier which is likely to have had an impact on the trendline.

To get a better view of the relationship, we will exclude the state with 19 stores from the dataframe and then re-plot to see a "truer relationship".

In [101]:
train_by_state_comp

,state,sum,count,avg_per_store
0,Pichincha,5.850674e+08,19,3.079302e+07
1,Guayas,1.651544e+08,11,1.501404e+07
2,Azuay,4.916860e+07,3,1.638953e+07
3,Tungurahua,4.030440e+07,2,2.015220e+07
4,Santo Domingo de los Tsachilas,3.583432e+07,3,1.194477e+07
5,El Oro,3.338680e+07,2,1.669340e+07
6,Los Rios,3.192537e+07,2,1.596269e+07
7,Manabi,2.496982e+07,3,8.323273e+06
8,Cotopaxi,2.118226e+07,2,1.059113e+07
9,Loja,1.886000e+07,1,1.886000e+07


In [102]:
# Visualizing the relationship between the number of stores and the total sales per state without the "outlier"
fig = px.scatter(train_by_state_comp[1:], x = "count", y = "sum", trendline = "ols",
                 title = "Relationship between number of stores and total sales per state",
             labels = {"state":"State", "sum":"Total Sales", "count":"Stores in State"})
fig.show()

From the plot, we may conclude that there is no relationship between the number of stores per state and the total sales from that state.

**What defines a store type?**

In [103]:
store_performance = train_data.groupby(by = "store_nbr").sales.agg(["sum"]).reset_index()
store_performance.head()

,store_nbr,sum
0,1,1.414501e+07
1,2,2.155739e+07
2,3,5.048191e+07
3,4,1.890970e+07
4,5,1.559240e+07


In [104]:
stores_data

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


In [105]:
store_details_with_performance = pd.merge(stores_data,store_performance)
store_details_with_performance.rename(columns = {"sum":"total_sales"}, inplace = True)
store_details_with_performance

,store_nbr,city,state,type,cluster,total_sales
0,1,Quito,Pichincha,D,13,1.414501e+07
1,2,Quito,Pichincha,D,13,2.155739e+07
2,3,Quito,Pichincha,D,8,5.048191e+07
3,4,Quito,Pichincha,D,9,1.890970e+07
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4,1.559240e+07
5,6,Quito,Pichincha,D,13,2.518368e+07
6,7,Quito,Pichincha,D,8,2.695205e+07
7,8,Quito,Pichincha,D,8,3.049429e+07
8,9,Quito,Pichincha,B,6,2.640944e+07
9,10,Quito,Pichincha,C,15,9.613906e+06


In [106]:
store_perf_by_type = store_details_with_performance.groupby("type").total_sales.agg(["count", "sum", "mean"]).reset_index()
store_perf_by_type

,type,count,sum,mean
0,A,9,3.530438e+08,3.922709e+07
1,B,8,1.452606e+08,1.815758e+07
2,C,15,1.644347e+08,1.096232e+07
3,D,18,3.510833e+08,1.950463e+07
4,E,4,5.982244e+07,1.495561e+07


From the table above, the nature of the relationship between the type of store and sales is unclear as type A which has 9 stores has more sales than Type D which had the most stores.

## Feature Engineering

**Merging all the dataframes**

In [107]:
# Taking a look at the training data before beginning the mergers
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6008904 entries, 0 to 6008903
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   store_nbr    int64  
 1   family       object 
 2   sales        float64
 3   onpromotion  float64
 4   sales_date   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 229.2+ MB


In [108]:
# Merging the train data with the other dataframes
merged_data = pd.merge(train_data, stores_data) # with the stores data
merged_data = pd.merge(merged_data, oil_data, left_on = "sales_date", right_on = "date") # with the oil data
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6008904 entries, 0 to 6008903
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   store_nbr    int64  
 1   family       object 
 2   sales        float64
 3   onpromotion  float64
 4   sales_date   object 
 5   city         object 
 6   state        object 
 7   type         object 
 8   cluster      int64  
 9   date         object 
 10  dcoilwtico   float64
dtypes: float64(3), int64(2), object(6)
memory usage: 550.1+ MB


In [109]:
# Ensuring same date formats with holiday data before merging
merged_data["sales_date"] = pd.to_datetime(merged_data["sales_date"]).dt.date
holidays_events["sales_date"] = pd.to_datetime(holidays_events["date"]).dt.date
holidays_events.drop(columns = ["date"], inplace = True)

In [110]:
holidays_events

,type,locale,locale_name,description,transferred,sales_date
0,Holiday,Local,Manta,Fundacion de Manta,False,2012-03-02
1,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,2012-04-01
2,Holiday,Local,Cuenca,Fundacion de Cuenca,False,2012-04-12
3,Holiday,Local,Libertad,Cantonizacion de Libertad,False,2012-04-14
4,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,2012-04-21
...,...,...,...,...,...,...
1777,Additional,National,Ecuador,Navidad-3,False,2017-12-22
1778,Additional,National,Ecuador,Navidad-2,False,2017-12-23
1779,Additional,National,Ecuador,Navidad-1,False,2017-12-24
1780,Holiday,National,Ecuador,Navidad,False,2017-12-25


In [111]:
holidays_events["locale_name"].value_counts()

Ecuador                           174
Quito                              13
Riobamba                           12
Guaranda                           12
Latacunga                          12
Ambato                             12
Guayaquil                          11
Cuenca                              7
Ibarra                              7
Salinas                             6
Loja                                6
Santa Elena                         6
Santo Domingo de los Tsachilas      6
Quevedo                             6
Manta                               6
Esmeraldas                          6
Cotopaxi                            6
El Carmen                           6
Santo Domingo                       6
Machala                             6
Imbabura                            6
Puyo                                6
Libertad                            6
Cayambe                             6
Name: locale_name, dtype: int64

In [112]:
# Merging with the holidays data
merged_data = pd.merge(merged_data, holidays_events, on = "sales_date")
merged_data

,store_nbr,family,sales,onpromotion,sales_date,city,state,type_x,cluster,date,dcoilwtico,type_y,locale,locale_name,description,transferred
0,1,AUTOMOTIVE,0.0,0.0,2013-01-01,Quito,Pichincha,D,13,2013-01-01,93.14,Holiday,National,Ecuador,Primer dia del ano,False
1,1,BABY CARE,0.0,0.0,2013-01-01,Quito,Pichincha,D,13,2013-01-01,93.14,Holiday,National,Ecuador,Primer dia del ano,False
2,1,BEAUTY,0.0,0.0,2013-01-01,Quito,Pichincha,D,13,2013-01-01,93.14,Holiday,National,Ecuador,Primer dia del ano,False
3,1,BEVERAGES,0.0,0.0,2013-01-01,Quito,Pichincha,D,13,2013-01-01,93.14,Holiday,National,Ecuador,Primer dia del ano,False
4,1,BOOKS,0.0,0.0,2013-01-01,Quito,Pichincha,D,13,2013-01-01,93.14,Holiday,National,Ecuador,Primer dia del ano,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6115819,9,POULTRY,0.0,0.0,2013-12-25,Quito,Pichincha,B,6,2013-12-25,98.87,Holiday,National,Ecuador,Navidad,False
6115820,9,PREPARED FOODS,0.0,0.0,2013-12-25,Quito,Pichincha,B,6,2013-12-25,98.87,Holiday,National,Ecuador,Navidad,False
6115821,9,PRODUCE,0.0,0.0,2013-12-25,Quito,Pichincha,B,6,2013-12-25,98.87,Holiday,National,Ecuador,Navidad,False
6115822,9,SCHOOL AND OFFICE SUPPLIES,0.0,0.0,2013-12-25,Quito,Pichincha,B,6,2013-12-25,98.87,Holiday,National,Ecuador,Navidad,False


In [113]:
# Looking at the nulls in the current version of the merged dataframe
merged_data.isnull().sum()

store_nbr            0
family               0
sales                0
onpromotion          0
sales_date           0
city                 0
state                0
type_x               0
cluster              0
date                 0
dcoilwtico           0
type_y         5103648
locale         5103648
locale_name    5103648
description    5103648
transferred    5103648
dtype: int64

We notice a large number of nulls, for the days that were not holidays so we replace with some values.

In [114]:
merged_data["type_y"].unique()

array(['Holiday', nan, 'Work Day', 'Additional', 'Event', 'Transfer',
       'Bridge'], dtype=object)

In [115]:
# Looking at the summary info of dates with type "Work Day"
zimbo = merged_data[merged_data["type_y"] == "Work Day"]
zimbo

,store_nbr,family,sales,onpromotion,sales_date,city,state,type_x,cluster,date,dcoilwtico,type_y,locale,locale_name,description,transferred
14256,1,AUTOMOTIVE,5.0,0.0,2013-01-05,Quito,Pichincha,D,13,2013-01-05,93.12,Work Day,National,Ecuador,Recupero puente Navidad,False
14257,1,BABY CARE,0.0,0.0,2013-01-05,Quito,Pichincha,D,13,2013-01-05,93.12,Work Day,National,Ecuador,Recupero puente Navidad,False
14258,1,BEAUTY,3.0,0.0,2013-01-05,Quito,Pichincha,D,13,2013-01-05,93.12,Work Day,National,Ecuador,Recupero puente Navidad,False
14259,1,BEVERAGES,1160.0,0.0,2013-01-05,Quito,Pichincha,D,13,2013-01-05,93.12,Work Day,National,Ecuador,Recupero puente Navidad,False
14260,1,BOOKS,0.0,0.0,2013-01-05,Quito,Pichincha,D,13,2013-01-05,93.12,Work Day,National,Ecuador,Recupero puente Navidad,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5110767,9,POULTRY,0.0,0.0,2016-11-12,Quito,Pichincha,B,6,2016-11-12,43.39,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False
5110769,9,PREPARED FOODS,0.0,0.0,2016-11-12,Quito,Pichincha,B,6,2016-11-12,43.39,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False
5110771,9,PRODUCE,0.0,0.0,2016-11-12,Quito,Pichincha,B,6,2016-11-12,43.39,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False
5110773,9,SCHOOL AND OFFICE SUPPLIES,0.0,0.0,2016-11-12,Quito,Pichincha,B,6,2016-11-12,43.39,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False


In [116]:
zimbo.nunique()

store_nbr        54
family           33
sales          2524
onpromotion      91
sales_date        5
city             22
state            16
type_x            5
cluster          17
date              5
dcoilwtico        5
type_y            1
locale            1
locale_name       1
description       5
transferred       1
dtype: int64

When we look at the summary data, we note that the locale value is national and transferred is false for all work days. This will be replicated across the merged dataframe to fill in the nulls for the missing dates which were not holidays.

In [117]:
# Filling the nulls in the holiday data
merged_data["type_y"] = merged_data["type_y"].fillna("Work Day")
merged_data["locale"] = merged_data["locale"].fillna("National")
merged_data["transferred"] = merged_data["transferred"].fillna(False)
merged_data.isnull().sum()

store_nbr            0
family               0
sales                0
onpromotion          0
sales_date           0
city                 0
state                0
type_x               0
cluster              0
date                 0
dcoilwtico           0
type_y               0
locale               0
locale_name    5103648
description    5103648
transferred          0
dtype: int64

In [118]:
# Merging with the transactions data
merged_data = pd.merge(merged_data, transactions, on = ["sales_date", "store_nbr"])
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5624718 entries, 0 to 5624717
Data columns (total 17 columns):
 #   Column        Dtype  
---  ------        -----  
 0   store_nbr     int64  
 1   family        object 
 2   sales         float64
 3   onpromotion   float64
 4   sales_date    object 
 5   city          object 
 6   state         object 
 7   type_x        object 
 8   cluster       int64  
 9   date          object 
 10  dcoilwtico    float64
 11  type_y        object 
 12  locale        object 
 13  locale_name   object 
 14  description   object 
 15  transferred   bool   
 16  transactions  float64
dtypes: bool(1), float64(4), int64(2), object(10)
memory usage: 734.9+ MB


In [119]:
# Dropping columns that will not be used
merged_data.drop(columns = ["date", "locale_name","description"], inplace = True)

# Renaming confusing column names
merged_data.rename(columns = {"type_x":"store_type", "type_y":"holiday_type", "dcoilwtico":"oil_price"}, inplace = True)
merged_data

,store_nbr,family,sales,onpromotion,sales_date,city,state,store_type,cluster,oil_price,holiday_type,locale,transferred,transactions
0,25,AUTOMOTIVE,0.0,0.0,2013-01-01,Salinas,Santa Elena,D,1,93.14,Holiday,National,False,770.0
1,25,BABY CARE,0.0,0.0,2013-01-01,Salinas,Santa Elena,D,1,93.14,Holiday,National,False,770.0
2,25,BEAUTY,2.0,0.0,2013-01-01,Salinas,Santa Elena,D,1,93.14,Holiday,National,False,770.0
3,25,BEVERAGES,810.0,0.0,2013-01-01,Salinas,Santa Elena,D,1,93.14,Holiday,National,False,770.0
4,25,BOOKS,0.0,0.0,2013-01-01,Salinas,Santa Elena,D,1,93.14,Holiday,National,False,770.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5624713,9,POULTRY,0.0,0.0,2013-12-25,Quito,Pichincha,B,6,98.87,Holiday,National,False,0.0
5624714,9,PREPARED FOODS,0.0,0.0,2013-12-25,Quito,Pichincha,B,6,98.87,Holiday,National,False,0.0
5624715,9,PRODUCE,0.0,0.0,2013-12-25,Quito,Pichincha,B,6,98.87,Holiday,National,False,0.0
5624716,9,SCHOOL AND OFFICE SUPPLIES,0.0,0.0,2013-12-25,Quito,Pichincha,B,6,98.87,Holiday,National,False,0.0


In [120]:
merged_data.nunique()

store_nbr           54
family              33
sales           379339
onpromotion        362
sales_date        1688
city                22
state               16
store_type           5
cluster             17
oil_price          994
holiday_type         6
locale               3
transferred          2
transactions      4994
dtype: int64

In [121]:
# Looking at the correlation between the variables in the merged dataframe
correlation = pd.DataFrame(merged_data.corr())
fig = px.imshow(correlation, text_auto = True, aspect = "auto")
fig.show()

In [122]:
# Removing the restriction on columns to display
pd.set_option('display.max_columns', None)

In [123]:
# Encoding the cities using one-hot encoding
encoded_cities = pd.get_dummies(merged_data.city)
encoded_cities.head()

,Ambato,Babahoyo,Cayambe,Cuenca,Daule,El Carmen,Esmeraldas,Guaranda,Guayaquil,Ibarra,Latacunga,Libertad,Loja,Machala,Manta,Playas,Puyo,Quevedo,Quito,Riobamba,Salinas,Santo Domingo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


Since cities are within the states, values can be aggregated by city to get the values for states, hence I will not include the state column in the features.

In [124]:
# Loading & applying the Label Encoder to the transferred column
label_encoder = preprocessing.LabelEncoder()
merged_data['family'] = label_encoder.fit_transform(merged_data.family)
merged_data['holiday_type'] = label_encoder.fit_transform(merged_data.holiday_type)
merged_data['locale'] = label_encoder.fit_transform(merged_data.locale)
merged_data['store_type'] = label_encoder.fit_transform(merged_data.store_type)
merged_data['transferred'] = label_encoder.fit_transform(merged_data.transferred)
merged_data

,store_nbr,family,sales,onpromotion,sales_date,city,state,store_type,cluster,oil_price,holiday_type,locale,transferred,transactions
0,25,0,0.0,0.0,2013-01-01,Salinas,Santa Elena,3,1,93.14,3,1,0,770.0
1,25,1,0.0,0.0,2013-01-01,Salinas,Santa Elena,3,1,93.14,3,1,0,770.0
2,25,2,2.0,0.0,2013-01-01,Salinas,Santa Elena,3,1,93.14,3,1,0,770.0
3,25,3,810.0,0.0,2013-01-01,Salinas,Santa Elena,3,1,93.14,3,1,0,770.0
4,25,4,0.0,0.0,2013-01-01,Salinas,Santa Elena,3,1,93.14,3,1,0,770.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5624713,9,28,0.0,0.0,2013-12-25,Quito,Pichincha,1,6,98.87,3,1,0,0.0
5624714,9,29,0.0,0.0,2013-12-25,Quito,Pichincha,1,6,98.87,3,1,0,0.0
5624715,9,30,0.0,0.0,2013-12-25,Quito,Pichincha,1,6,98.87,3,1,0,0.0
5624716,9,31,0.0,0.0,2013-12-25,Quito,Pichincha,1,6,98.87,3,1,0,0.0


In [125]:
# Creating the final training dataframe
final_train = merged_data.join(encoded_cities).drop(columns = ["city", "state"])
final_train.head()

,store_nbr,family,sales,onpromotion,sales_date,store_type,cluster,oil_price,holiday_type,locale,transferred,transactions,Ambato,Babahoyo,Cayambe,Cuenca,Daule,El Carmen,Esmeraldas,Guaranda,Guayaquil,Ibarra,Latacunga,Libertad,Loja,Machala,Manta,Playas,Puyo,Quevedo,Quito,Riobamba,Salinas,Santo Domingo
0,25,0,0.0,0.0,2013-01-01,3,1,93.14,3,1,0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,25,1,0.0,0.0,2013-01-01,3,1,93.14,3,1,0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,25,2,2.0,0.0,2013-01-01,3,1,93.14,3,1,0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,25,3,810.0,0.0,2013-01-01,3,1,93.14,3,1,0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,25,4,0.0,0.0,2013-01-01,3,1,93.14,3,1,0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [126]:
final_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5624718 entries, 0 to 5624717
Data columns (total 34 columns):
 #   Column         Dtype  
---  ------         -----  
 0   store_nbr      int64  
 1   family         int32  
 2   sales          float64
 3   onpromotion    float64
 4   sales_date     object 
 5   store_type     int32  
 6   cluster        int64  
 7   oil_price      float64
 8   holiday_type   int32  
 9   locale         int32  
 10  transferred    int64  
 11  transactions   float64
 12  Ambato         uint8  
 13  Babahoyo       uint8  
 14  Cayambe        uint8  
 15  Cuenca         uint8  
 16  Daule          uint8  
 17  El Carmen      uint8  
 18  Esmeraldas     uint8  
 19  Guaranda       uint8  
 20  Guayaquil      uint8  
 21  Ibarra         uint8  
 22  Latacunga      uint8  
 23  Libertad       uint8  
 24  Loja           uint8  
 25  Machala        uint8  
 26  Manta          uint8  
 27  Playas         uint8  
 28  Puyo           uint8  
 29  Quevedo       

In [129]:
# Defining a function to get date features from dataframe
def getDateFeatures(df, date):
    df['date'] = pd.to_datetime(df[date])
    df['day_of_week'] = df['date'].dt.dayofweek.astype(int)
    df['day_of_month'] = df['date'].dt.day.astype(int)
    df['day_of_year'] = df['date'].dt.dayofyear.astype(int)
    df['is_weekend'] = np.where(df['day_of_week'] > 4, 1, 0).astype(int)
    
    df['week_of_year'] = df['date'].dt.isocalendar().week.astype(int)
    
    df['month'] = df['date'].dt.month.astype(int)
    df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
    
    df['quarter'] = df['date'].dt.quarter.astype(int)
    df['is_quarter_start'] = df['date'].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df['date'].dt.is_quarter_end.astype(int)
    
    df['is_year_start'] = df['date'].dt.is_year_end.astype(int)
    df['is_year_end'] = df['date'].dt.is_year_end.astype(int)
    df['year'] = df['date'].dt.year.astype(int)
    df = df.drop(columns = "date")
    
    return df

In [130]:
# Getting the date features from the Train Dataframe
final_train = getDateFeatures(final_train, "sales_date")
final_train = final_train.sort_values(by = ["sales_date", "store_nbr"], ignore_index = True)
final_train

,store_nbr,family,sales,onpromotion,sales_date,store_type,cluster,oil_price,holiday_type,locale,transferred,transactions,Ambato,Babahoyo,Cayambe,Cuenca,Daule,El Carmen,Esmeraldas,Guaranda,Guayaquil,Ibarra,Latacunga,Libertad,Loja,Machala,Manta,Playas,Puyo,Quevedo,Quito,Riobamba,Salinas,Santo Domingo,day_of_week,day_of_month,day_of_year,is_weekend,week_of_year,month,is_month_start,is_month_end,quarter,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year
0,25,0,0.0,0.0,2013-01-01,3,1,93.14,3,1,0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,1,0,0,0,2013
1,25,1,0.0,0.0,2013-01-01,3,1,93.14,3,1,0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,1,0,0,0,2013
2,25,2,2.0,0.0,2013-01-01,3,1,93.14,3,1,0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,1,0,0,0,2013
3,25,3,810.0,0.0,2013-01-01,3,1,93.14,3,1,0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,1,0,0,0,2013
4,25,4,0.0,0.0,2013-01-01,3,1,93.14,3,1,0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,1,1,0,1,1,0,0,0,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5624713,54,28,0.0,0.0,2017-08-15,2,3,47.57,3,0,0,802.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,15,227,0,33,8,0,0,3,0,0,0,0,2017
5624714,54,29,0.0,0.0,2017-08-15,2,3,47.57,3,0,0,802.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,15,227,0,33,8,0,0,3,0,0,0,0,2017
5624715,54,30,0.0,0.0,2017-08-15,2,3,47.57,3,0,0,802.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,15,227,0,33,8,0,0,3,0,0,0,0,2017
5624716,54,31,0.0,0.0,2017-08-15,2,3,47.57,3,0,0,802.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,15,227,0,33,8,0,0,3,0,0,0,0,2017


In [131]:
# Looking at the correlation between the variables in the final dataframe
correlation = pd.DataFrame(final_train.corr())
fig = px.imshow(correlation, text_auto = True, aspect = "auto")
fig.show()

## Modelling

In [132]:
train = final_train.loc[final_train["year"].isin([2013, 2014, 2015, 2016])]
validation = final_train.loc[final_train["year"].isin([2017])]

In [133]:
# Training data
X = train[["store_nbr", "family", "onpromotion", "store_type", "cluster", "oil_price", "day_of_week", 
           "day_of_month", "day_of_year", "is_weekend", "week_of_year", "month", "year"]]
y = train["sales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(3609094, 13) (3609094,)
(1203032, 13) (1203032,)
